In [15]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import math
import os
import sys
import warnings
import argparse
import numpy as np
import yaml
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import csv
import torch.nn as nn
import torch.backends.cudnn as cudnn
import random

from PIL import Image,ImageFilter
import PIL.ImageOps as imop
from glob import glob
from tqdm import tqdm
from torch import optim
from torch import Tensor
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset,Sampler
from torchvision import datasets,transforms
from torch.nn import functional as F

sys.path.append("/home/shiyi/gpu/gpu/")
img_dir = "/data/colposcopy/colposcopy"
img_dir_1 = "/data/colposcopy/colposcopy_1/"

from gpu_allocation import set_gpu

error = glob(os.path.join(img_dir,"train*","*","2845.jpg")) + \
    glob(os.path.join(img_dir,"train*","*","5893.jpg")) + glob(os.path.join(img_dir,"train*","*","5892.jpg"))
print(error)

from gpu_allocation import set_gpu
corrupt_set  = set()
types = [1,2,3]

type_1_train = [i for i in glob(os.path.join(img_dir,"train","Type_1","*.jpg")) if i not in error]
type_2_train = [i for i in glob(os.path.join(img_dir,"train","Type_2","*.jpg")) if i not in error]
type_3_train = [i for i in glob(os.path.join(img_dir,"train","Type_3","*.jpg")) if i not in error]

type_1_train_extra = [i for i in glob(os.path.join(img_dir,"train_stg2","Type_1","*.jpg")) if i not in error]
type_2_train_extra = [i for i in glob(os.path.join(img_dir,"train_stg2","Type_2","*.jpg")) if i not in error]
type_3_train_extra = [i for i in glob(os.path.join(img_dir,"train_stg2","Type_3","*.jpg")) if i not in error]

img_test   = glob(os.path.join(img_dir,"test","*.jpg"))
img_test_wo_answer = glob(os.path.join(img_dir,"test_stg2","*.jpg"))

img_path_yolo = type_1_train+type_2_train+type_3_train
img_paths = img_path_yolo+img_test+img_test_wo_answer+type_1_train_extra+type_2_train_extra+type_3_train_extra
#print(len(img_paths))
#print(img_paths[0].split('/')[4])
print(len(img_path_yolo))

def data_label_alter(csv_path):
    
    def data_label_del(target_img_path):
        for i in img_path_yolo:
            if i.split('/')[-1]==target_img_path:
                img_path_yolo.remove(i)
                #print ("%s removed"%i)
                return
                
    def data_label_exchange(target_img_path,src,target):
        for i in range(len(img_path_yolo)):
            path_token = img_path_yolo[i].split('/')
            if path_token[-1]==target_img_path and src == path_token[-2]:
                #print (target_img_path,src,target)
                #print (path_token[-2],src,target)
                path_token[-2] = target
               #print ("From",img_paths[i])
                img_path_yolo[i] = '/'.join(path_token)
               # print ("To",img_paths[i])
                return 
                
    with open(csv_path) as f:
        f_csv = csv.reader(f)
        index = 0
        for row in f_csv:
            if index == 0:
                index += 1
                continue
            if row[-1] == 'remove':
                data_label_del(row[0])
            else:
                data_label_exchange(row[0],row[1],row[2])

#data_label_alter(os.path.join(img_dir,"fixed_labels_v2.csv"))
data_label_alter(os.path.join(img_dir,"removed_files.csv"))

print(len(img_path_yolo))
print(len(type_1_train),len(type_2_train),len(type_3_train))
img_path_yolo = map(lambda x:str(x),img_path_yolo)
#print(img_path_yolo)

[u'/data/colposcopy/colposcopy/train_stg2/Type_1/5893.jpg']
1481
1409
250 781 450


In [16]:
!cat /home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/787_extra_2.txt

cat: /home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/787_extra_2.txt: No such file or directory


In [17]:
class ColpoProcessDataset(Dataset):
    def __init__(self,paths):
        self.paths = paths
    def __getitem__(self, index):
        path = str(self.paths[index])
        #print (path)
        return path
    def __len__(self):
        return len(self.paths)
    
def cvt(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def transform_offline(img):
    pass
def write_to_dir(img):
    pass

#added_data = []
DataYolo = ColpoProcessDataset(img_path_yolo)
LoaderYolo = DataLoader(dataset = DataYolo, batch_size = 32, num_workers = 8)
index = 0

#transf = [transforms.RandomHorizontalFlip(),transforms.ColorJitter(brightness=0.1)]
#distort = transforms.Compose(transf)
path_parent = "/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom"
img_parent = "/data/colposcopy/colposcopy/train"

error_list = []
for batch in tqdm(LoaderYolo):
    for path in batch:
        cur_name = path.split('/')[-1].split('.')[0]
        cur_type = path.split('/')[-2]
        if cur_type == 'Type_2':
            continue
            
        label_path_base =  os.path.join(path_parent,'labels',cur_name+'_extra')
        img_path_base =  os.path.join(img_parent,cur_type, cur_name+'_extra')
        try:
            f = open(os.path.join(path_parent,'labels',cur_name+'.txt'))
            label_coords = f.readlines()[0]
            f.close()
        except IOError:
            error_list.append(os.path.join(path_parent,'labels',cur_name+'.txt'))
            continue
        
        img = Image.open(path)
           
        if cur_type == 'Type_1' or (cur_type == 'Type_3' and random.randint(0,2)<=1) :
            #plt.figure(figsize=(15,15))
            #plt.subplot(131)
            #plt.imshow(img)
            img = img.filter(ImageFilter.GaussianBlur(radius=5)) 
            img.save(img_path_base+'_1.jpg')
            f = open(label_path_base+'_1.txt','w')
            f.write(label_coords)
            f.close()

        if cur_type == 'Type_1':
            img2 = img.transpose(Image.FLIP_LEFT_RIGHT) #distort(img) 
            img.save(img_path_base+'_2.jpg')
            label_coords = [eval(i) for i in label_coords.strip('\n').split()]
            #print (label_coords)
            label_coords[1] = 1-label_coords[1] 
            label_coords = [str(i) for i in label_coords]
            
            #print(label_coords)
            f = open(label_path_base+'_2.txt','w')
            f.write(' '.join(label_coords)+'\n')
            f.close()
print (len(error_list),error_list)


100%|██████████| 45/45 [13:35<00:00, 18.12s/it]

6 [u'/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1339.txt', u'/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/124.txt', u'/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1307.txt', u'/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1434.txt', u'/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1431.txt', u'/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/980.txt']


In [42]:
!cat /home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/train.txt | grep 630_extra_1.jpg
!ls /data/colposcopy/colposcopy/train/Type_3 | grep 630_extra_1.jpg
!cat /home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/630_extra_1.txt

cat: /home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/630_extra_1.txt: No such file or directory


In [6]:
from sklearn.model_selection import train_test_split
from glob import glob
import os
img_dir = "/data/colposcopy/colposcopy"
type_1_train = [i for i in glob(os.path.join(img_dir,"train","Type_1","*.jpg"))]
type_2_train = [i for i in glob(os.path.join(img_dir,"train","Type_2","*.jpg"))]
type_3_train = [i for i in glob(os.path.join(img_dir,"train","Type_3","*.jpg"))]
test = [i for i in glob(os.path.join(img_dir,"test","*.jpg"))]
label_paths = [i for i in glob("/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/*")]

file_paths = type_1_train+type_2_train+type_3_train+test
#print(len(type_1_train),len(type_2_train),len(type_3_train))
#print (type_1_train)
train,valid,_,_ = train_test_split(file_paths,file_paths,test_size=0.2, random_state=42)
#print (train,valid)
yolo_train_table = "/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/train.txt"
yolo_valid_table = "/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/valid.txt"
remove_table_path = "/data/colposcopy/colposcopy/removed_files.csv"

f0 = open(remove_table_path,'r')
rm_lst = f0.readlines()[1:]
rm_lst = map(lambda x: str(x.strip().split(',')[0]),rm_lst)
#print (rm_lst)
f0.close()

path_parent = "/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels"
label_lst = map(lambda x : str(x.split('/')[-1].split('.')[0]), glob(os.path.join(path_parent,"*.txt")))

with open(yolo_train_table,'w') as f:
    for i in train:
        name = i.split('/')[-1].split('.')[0]
        label_path = os.path.join(path_parent,name+'.txt')
        print(label_path)
        if i.split('/')[-1] in rm_lst or not os.path.exists(label_path) or name not in label_lst:
            continue
        
        f.write(i+'\n')
        
with open(yolo_valid_table,'w') as f:
    for i in valid:
        name = i.split('/')[-1].split('.')[0]
        label_path = os.path.join(path_parent,name+'.txt')
        
        if i.split('/')[-1] in rm_lst or not os.path.exists(label_path) or name not in label_lst:
            continue
    
        f.write(i+'\n')


/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1081.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/614.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1060.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/660_extra_1.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1088.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1150.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/130.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1319.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/986.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/145.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/447_test.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/l

/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/580_extra_1.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1472.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/532_extra_1.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/545.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/43_test.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1307.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/532_extra_2.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/367.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/826.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/160.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/233_test.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOL

/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/797.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/280.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/829.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/257_test.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/125_test.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/176.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/212_extra_1.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/441_test.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/262_extra_2.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1174.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1455_extra_1.txt
/home/shiyi/colposcopy/colposcopy/models/PyTor

/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/376_extra_1.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1235_extra_1.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/342_test.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/518_extra_2.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/231.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/791.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/217.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/69.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/518.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/77_test.txt
/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/labels/1056_extra_1.txt
/home/shiyi/colposcopy/colposcopy/models/PyTo

In [37]:
f = open("/home/shiyi/colposcopy/colposcopy/models/PyTorch-YOLOv3/data/custom/train.txt",'r')
names = f.readlines()
print(names)
for i in names:
    if not os.path.exists(i):
        continue
    print (i)
    

['/data/colposcopy/colposcopy/train/Type_2/1178.jpg\n', '/data/colposcopy/colposcopy/train/Type_2/827.jpg\n', '/data/colposcopy/colposcopy/train/Type_3/1081.jpg\n', '/data/colposcopy/colposcopy/train/Type_3/318.jpg\n', '/data/colposcopy/colposcopy/train/Type_3/1069.jpg\n', '/data/colposcopy/colposcopy/train/Type_2/1039.jpg\n', '/data/colposcopy/colposcopy/train/Type_2/618.jpg\n', '/data/colposcopy/colposcopy/train/Type_3/277_extra_1.jpg\n', '/data/colposcopy/colposcopy/train/Type_2/687.jpg\n', '/data/colposcopy/colposcopy/train/Type_1/1204_extra_1.jpg\n', '/data/colposcopy/colposcopy/train/Type_3/1277_extra_1.jpg\n', '/data/colposcopy/colposcopy/train/Type_2/533.jpg\n', '/data/colposcopy/colposcopy/train/Type_3/1458.jpg\n', '/data/colposcopy/colposcopy/train/Type_3/196_extra_1.jpg\n', '/data/colposcopy/colposcopy/train/Type_3/692_extra_1.jpg\n', '/data/colposcopy/colposcopy/train/Type_2/483.jpg\n', '/data/colposcopy/colposcopy/train/Type_1/1134_extra_2.jpg\n', '/data/colposcopy/colposc

In [41]:
print(len(img_paths))
for img_path in tqdm(img_paths):
    img_path_target = os.path.join(img_dir_1 + '/'.join(img_path.split('/')[4:]))
    img = cv2.imread(img_path)
    if img is None:
        continue
            #img = preprocess(img)
            #print("Thread {}, number {}".format(self.threadID,index))
    cv2.imwrite(img_path_target,img,[int(cv2.IMWRITE_JPEG_QUALITY),100])

def preprocess(img): 
    r_c = img[:,:,2]
    g_c = img[:,:,1]
    b_c = img[:,:,0]
    _,r_thres = cv2.threshold(r_c, 210, 255, cv2.THRESH_BINARY)
    _,g_thres = cv2.threshold(g_c, 208, 255, cv2.THRESH_BINARY)
    _,b_thres = cv2.threshold(b_c, 206, 255, cv2.THRESH_BINARY)
    mixed = cv2.UMat(r_thres & b_thres & g_thres)
    
    kernel_dilate = np.ones((18, 18), np.uint8)
    mask = cv2.dilate(mixed,kernel_dilate)
    dst = cv2.inpaint(img,mask,14,cv2.INPAINT_TELEA)
    return dst

import threading
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"
num_gpu = 1
set_gpu(num_gpu)

class myThread (threading.Thread):   
    def __init__(self, threadID, img_num_per_thread):
        threading.Thread.__init__(self)
        self.threadID = threadID
        self.img_num_per_thread = img_num_per_thread
        
    def run(self):     
        start = self.img_num_per_thread*self.threadID
        end = min(start+self.img_num_per_thread,len(img_paths))
        index = 0
        for img_path in tqdm(img_paths[start:end]):
            img_path_target = os.path.join(img_dir_1 + '/'.join(img_path.split('/')[4:]))
            index += 1
            img = cv2.imread(img_path)
            if img is None:
                continue
            #img = preprocess(img)
            #print("Thread {}, number {}".format(self.threadID,index))
            cv2.imwrite(img_path_target,img,[int(cv2.IMWRITE_JPEG_QUALITY),100])

num_thread = num_gpu
img_num_per_thread = len(img_paths)//num_thread 
thread_res = list()
for i in range(num_thread):
    thread_res.append(myThread(i,img_num_per_thread))

for i in range(num_thread): 
    thread_res[i].start()

    

Thread 0, number 61


  0%|          | 20/12153 [00:09<1:37:44,  2.07it/s]

Thread 0, number 20
('Found 4 GPU(s)', 'only 2 gpu below threshold')
Using GPU 1



  0%|          | 0/12153 [00:00<?, ?it/s]

Thread 0, number 62
Thread 0, number 21


...

  0%|          | 36/12153 [00:15<1:22:24,  2.45it/s]

Thread 0, number 73
Thread 0, number 36



  0%|          | 14/12153 [00:06<1:31:35,  2.21it/s]

Thread 0, number 37
Thread 0, number 74


  0%|          | 37/12153 [00:16<1:40:24,  2.01it/s]

In [5]:
import numpy as np
import math

def max2D(matrix):
    def max_with_par(val,p1,p2,max_val,max_p1,max_p2):
        if val>max_val:
            return val,p1,p2
        else:
            return max_val,max_p1,max_p2
        
    def maxHist(lst):
        if not lst:
            return 0,0,0,0
        
        s = list()
        max_val = 0
        left = -1
        right = -1
        i = 0
        while(i < len(lst)):
            if len(s)==0 or lst[i]>=lst[s[-1]]:
                s.append(i)
                i += 1
            else:
                top = s.pop()
                val = lst[top] * (i if len(s)<=0 else i-s[-1]-1)
                left_cur = 0 if len(s)<=0 else s[-1]+1
                right_cur = i-1
                max_val,left,right = max_with_par(val,left_cur,right_cur,max_val,left,right)
    
        while(len(s) > 0):
            top = s.pop()
            val = lst[top] * (i if len(s)<=0 else i-s[-1]-1)
            left_cur = 0 if len(s)<=0 else s[-1]+1
            right_cur = i-1
            max_val,left,right = max_with_par(val,left_cur,right_cur,max_val,left,right)
        return max_val,left,right
    
    max_area = -1
    up = -1
    down = -1
    left = -1
    right = -1
    
    row_accum =[int(i) for i in matrix[0]]
    for index in range(len(matrix)):
        if index>=1:
            for digit in range(len(matrix[index-1])):
                if int(matrix[index][digit])==0:
                    row_accum[digit] = 0
                else:
                    row_accum[digit] += int(matrix[index][digit]) 
 
        area,left_cur,right_cur = maxHist(row_accum)
        past_max_area = max_area
        
        down_cur = index
        up_cur = index - (area//(1-left_cur+right_cur))+1
        max_area,down,up = max_with_par(area,down_cur,up_cur,max_area,down,up)
        if past_max_area != max_area:
            left = left_cur
            right = right_cur
        #print('area: ',max_area,area,'  coor: ',left,right,down,up)
    return left,up,right,down

In [6]:
maxtrix_0 = np.array([[1,0,0,0,0,1],
                      [1,1,0,1,1,1],
                      [0,0,0,1,1,1],
                      [0,0,1,1,0,0]])

maxtrix_1 = np.array([[1,0,1,1,1,1],
                      [1,1,1,1,1,0],
                      [0,0,1,1,1,1],
                      [0,0,1,1,0,0]])

print (max2D(maxtrix_0))
print (max2D(maxtrix_1))

(3, 1, 5, 2)
(2, 0, 4, 2)


In [8]:
from sklearn.mixture import GaussianMixture 
from sklearn.utils import shuffle
from skimage import measure

def cvt(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def cropCircle(img):
    '''
    max_edge = 255
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*max_edge/img.shape[0]),max_edge)
    else:
        tile_size = (max_edge, int(img.shape[0]*max_edge/img.shape[1]))
    '''
    tile_size = (255,255)

    img = cv2.resize(img, dsize=tile_size)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)
    _, contours, _ = cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    main_contour = sorted(contours, key = cv2.contourArea, reverse = True)[0]
            
    ff = np.zeros((gray.shape[0],gray.shape[1]), 'uint8') 
    cv2.drawContours(ff, main_contour, -1, 1, 15)
    ff_mask = np.zeros((gray.shape[0]+2,gray.shape[1]+2), 'uint8')
    cv2.floodFill(ff, ff_mask, (int(gray.shape[1]/2), int(gray.shape[0]/2)), 1)
    rect = max2D(ff)
    
    rectangle = [min(rect[0],rect[2]), max(rect[0],rect[2]), min(rect[1],rect[3]), max(rect[1],rect[3])]
    return img[rectangle[0]:rectangle[1], rectangle[2]:rectangle[3]]
    
def remove_sr(img): 
    r_c = img[:,:,2]
    g_c = img[:,:,1]
    b_c = img[:,:,0]
    _,r_thres = cv2.threshold(r_c, 210, 255, cv2.THRESH_BINARY)
    _,g_thres = cv2.threshold(g_c, 208, 255, cv2.THRESH_BINARY)
    _,b_thres = cv2.threshold(b_c, 206, 255, cv2.THRESH_BINARY)
    mixed = r_thres & b_thres & g_thres
    kernel_dilate = np.ones((18, 18), np.uint8)
    mask = cv2.dilate(mixed,kernel_dilate)
    dst = cv2.inpaint(img,mask,14,cv2.INPAINT_TELEA)
    return dst

In [14]:
from multiprocessing import Pool, Lock
from multiprocessing.pool import ThreadPool
import time
import threading
from tqdm import tqdm
import os
import numpy as np

def preprocess_cervical(thread_id,index_slice):
    for index in index_slice:
        if (index-index_slice[0])%40==0:
            print("Thread {}, image {}|{}, name: {}".format(thread_id, index-index_slice[0], len(index_slice) ,img_paths[index]))
        img_name = img_paths[index]
        img_path_target = os.path.join(img_dir_1 + '/'.join(img_path.split('/')[4:]))
        img = cv2.imread(img_name)
        if img is not None:
            img_crop = cropCircle(cvt(img))
            dst = remove_sr(img_crop)
            cv2.imwrite(img_name_target,dst)
    
print ('Parent process %s.' % os.getpid())

num_thread = 10
num_images = len(img_paths)
pool = ThreadPool(num_thread)
num_per_thread = num_images//num_thread


start_time = time.time()
for i in range(num_thread):
    start = i*num_per_thread
    end = min((i+1)*num_per_thread, num_images)
    pool.apply_async(preprocess_cervical, args=(i,range(start,end)))

print ('Waiting for all subprocesses done...')

pool.close()
pool.join()
end_time = time.time()
print ("done,time takes:",end_time-start_time)


Parent process 11247.
1215 10
Thread 0, image 0|1215, name: /data/colposcopy/colposcopy/train/Type_1/732.jpg
Thread 1, image 0|1215, name: /data/colposcopy/colposcopy/train_stg2/Type_1/3829.jpg
Thread 2, image 0|1215, name: /data/colposcopy/colposcopy/train_stg2/Type_2/6430.jpg
Thread 3, image 0|1215, name: /data/colposcopy/colposcopy/train_stg2/Type_2/780.jpg
Waiting for all subprocesses done...
Thread 4, image 0|1215, name: /data/colposcopy/colposcopy/train_stg2/Type_2/919.jpg
Thread 5, image 0|1215, name: /data/colposcopy/colposcopy/train/Type_3/818.jpg
Thread 6, image 0|1215, name: /data/colposcopy/colposcopy/train_stg2/Type_3/4423.jpg
Thread 7, image 0|1215, name: /data/colposcopy/colposcopy/test/202.jpg
Thread 8, image 0|1215, name: /data/colposcopy/colposcopy/test_stg2/11193.jpg
Thread 9, image 0|1215, name: /data/colposcopy/colposcopy/test_stg2/10258.jpg
done,time takes: 0.101037025452


In [ ]:
from multiprocessing import Lock
from multiprocessing.dummy import Pool as ThreadPool 
import time
import threading
from tqdm import tqdm
import os
import numpy as np

num_thread = 20
num_images = len(img_paths)


pool = ThreadPool(num_thread)
progress = tqdm(total=num_images)

def preprocess_cervical_1(name):
    global progress,lock
    #print("Thread {}, image {}|{}, name: {}".format(thread_id, index-index_slice[0], len(index_slice) ,img_paths[index]))
    img_name_target = os.path.join(img_dir_1 + '/'.join(name.split('/')[4:]))
    print (name)
    try:
        img = cv2.imread(name)
        if img is not None:
            img_crop = cropCircle(cvt(img))
            dst = remove_sr(img_crop)
            cv2.imwrite(img_name_target,dst)
    except cv2.error:
        print ("skipped,continue")
   
        
print ('Parent process %s.' % os.getpid())
start_time = time.time()
a = pool.map(preprocess_cervical_1, img_paths)
pool.close()
pool.join()
end_time = time.time()

print ("done,it takes:",end_time-start_time)




  0%|          | 0/12153 [00:00<?, ?it/s]

Parent process 12852.
/data/colposcopy/colposcopy/train/Type_1/732.jpg
/data/colposcopy/colposcopy/train/Type_1/624.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/174.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3041.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1185.jpg/data/colposcopy/colposcopy/train_stg2/Type_1/2054.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/6460.jpg

/data/colposcopy/colposcopy/train_stg2/Type_1/1877.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/4023.jpg/data/colposcopy/colposcopy/train_stg2/Type_1/5982.jpg
/data/colposcopy/colposcopy/train/Type_2/1298.jpg/data/colposcopy/colposcopy/train/Type_2/974.jpg

/data/colposcopy/colposcopy/train/Type_2/723.jpg
/data/colposcopy/colposcopy/train/Type_2/903.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4329.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/5363.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/2109.jpg
/data/colposcopy/colposcopy/train/Type_2/865.jpg
/data/colposcopy/colposcopy/tra

/data/colposcopy/colposcopy/train_stg2/Type_1/1642.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/6861.jpg
/data/colposcopy/colposcopy/train/Type_2/644.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/6329.jpg
/data/colposcopy/colposcopy/train/Type_2/729.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4136.jpg
/data/colposcopy/colposcopy/train/Type_2/1006.jpg
/data/colposcopy/colposcopy/train/Type_1/469.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1214.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2235.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/5995.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1676.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2434.jpg
/data/colposcopy/colposcopy/train/Type_2/175.jpg
/data/colposcopy/colposcopy/train/Type_1/779.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/3521.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3673.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4220.jpg
/data/colposcopy/colposcopy/train/Type_2/1443

In [ ]:
def cvt(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def preprocess(img): 
    r_c = img[:,:,2]
    g_c = img[:,:,1]
    b_c = img[:,:,0]
    _,r_thres = cv2.threshold(r_c, 210, 255, cv2.THRESH_BINARY)
    _,g_thres = cv2.threshold(g_c, 208, 255, cv2.THRESH_BINARY)
    _,b_thres = cv2.threshold(b_c, 206, 255, cv2.THRESH_BINARY)
    mixed = cv2.UMat(r_thres & b_thres & g_thres)
    
    kernel_dilate = np.ones((18, 18), np.uint8)
    mask = cv2.dilate(mixed,kernel_dilate)
    dst = cv2.inpaint(img,mask,14,cv2.INPAINT_TELEA)
    return dst

def maxRect(matrix):
    def max_with_par(val,p1,p2,max_val,max_p1,max_p2):
        if val>max_val:
            return val,p1,p2
        else:
            return max_val,max_p1,max_p2
        
    def maxHist(lst):
        s = list()
        max_val = 0
        left = -1
        right = -1
        i = 0
        while(i < len(lst)):
            if len(s)==0 or lst[i]>=lst[s[-1]]:
                s.append(i)
                i += 1
            else:
                top = s.pop()
                val = lst[top] * (i if len(s)<=0 else i-s[-1]-1)
                left_cur = 0 if len(s)<=0 else s[-1]
                right_cur = i-1
                max_val,left,right = max_with_par(val,left_cur,right_cur,max_val,left,right)
    
        while(len(s) > 0):
            top = s.pop()
            val = lst[top] * (i if len(s)<=0 else i-s[-1]-1)
            left_cur = 0 if len(s)<=0 else s[-1]
            right_cur = i-1
            max_val,left,right = max_with_par(val,left_cur,right_cur,max_val,left,right)
        return max_val,left,right
    
    max_area = -1
    up = -1
    down = -1
    left = -1
    right = -1
    
    row_accum =[int(i) for i in matrix[0]]
    for index in range(len(matrix)):
        if index>=1:
            for digit in range(len(matrix[index-1])):
                if int(matrix[index][digit])==0:
                    row_accum[digit] = 0
                else:
                    row_accum[digit] += int(matrix[index][digit]) 
 
        area,cur_left,cur_right = maxHist(row_accum)
        past_max_area = max_area
        cur_down = index
        cur_up = int(index-math.floor(area/(right-left+1)))
        max_area,down,up = max_with_par(area,cur_down,cur_up,max_area,down,up)
        if past_max_area != max_area:
            left = cur_left
            right = cur_right
    return [left,up,right,down]
   
def preprocess(matrix):
    print(maxRect(matrix))

if __name__ == '__main__':
    matrix = np.array([[1,0,0,0,0,1],
                       [1,1,0,1,1,1],
                       [0,0,0,1,1,1],
                       [0,0,1,1,0,0]])
    threads = []

    for i in range(3):
        t1 = threading.Thread(target=preprocess,args=(matrix,))
        threads.append(t1)
    
    for t in threads:
        t.setDaemon(True)
        t.start()
        
    t.join()
    print ("all over %s" %ctime())

In [50]:
import piexif
test_path = glob("/data/colposcopy/colposcopy/train_*/*/*.jpg")
print(len(test_path))
for i in test_path:
    print(i)
    piexif.remove(i)

6732
/data/colposcopy/colposcopy/train_stg2/Type_2/3093.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/5387.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/318.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1720.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/3957.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4055.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4632.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/6663.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/6478.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/3910.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/2600.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/422.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/5602.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/732.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1400.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/3809.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4246.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/5348.jpg
/data/co

/data/colposcopy/colposcopy/train_stg2/Type_2/999.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1830.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4528.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1194.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4672.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/3982.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/735.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/6248.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1118.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/6616.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/6250.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4233.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/3465.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/6855.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/630.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/559.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1210.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4584.jpg
/data/colposco

/data/colposcopy/colposcopy/train_stg2/Type_2/2174.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1486.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4621.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/786.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1314.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/2334.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1639.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1208.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4253.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/3283.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/6823.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4426.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4593.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1099.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/2510.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/592.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1984.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/5745.jpg
/data/colpos

/data/colposcopy/colposcopy/train_stg2/Type_2/5540.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/426.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/3967.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4728.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/5469.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/6041.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1006.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4299.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1494.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/6518.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/2355.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/5654.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1752.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4596.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/405.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/2889.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4062.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1817.jpg
/data/colpos

/data/colposcopy/colposcopy/train_stg2/Type_2/3051.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/869.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4900.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/2104.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1991.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4894.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1168.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/5970.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4911.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/2955.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/3113.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/2611.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/55.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/4567.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/3248.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/1389.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/6849.jpg
/data/colposcopy/colposcopy/train_stg2/Type_2/3943.jpg
/data/colposc

...

/data/colposcopy/colposcopy/train_stg2/Type_1/2463.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3442.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1339.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3981.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/4065.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1304.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2519.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2032.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1930.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1200.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3058.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1370.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/6369.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2028.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2203.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2876.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3974.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/262.jpg
/data/colpo

/data/colposcopy/colposcopy/train_stg2/Type_1/2417.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2169.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3331.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3111.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2967.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/6782.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/817.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2368.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3048.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2829.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3472.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/5848.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/6655.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2738.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3464.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/4323.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/4005.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/6918.jpg
/data/colpo

/data/colposcopy/colposcopy/train_stg2/Type_1/3141.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/505.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3755.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/731.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/10.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/5358.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1706.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1011.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1491.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/6102.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2862.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1047.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/5946.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/493.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2356.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/6374.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1307.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/43.jpg
/data/colposcopy/

/data/colposcopy/colposcopy/train_stg2/Type_1/3023.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/625.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2367.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2237.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3637.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2057.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/5355.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/2622.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3504.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/6101.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1054.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/4473.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/1117.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/4540.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/943.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/4011.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/6654.jpg
/data/colposcopy/colposcopy/train_stg2/Type_1/3349.jpg
/data/colpos

UnboundLocalError: local variable 'file_type' referenced before assignment